In [11]:
import pandas as pd
import rpy2.robjects as robjects
import os

In [2]:
df = pd.read_csv("/Users/mrobredo23/OULU/PROMISE_TDpred2024/data/complete_data/archiva.csv")
df

,SQALE_INDEX,COMMIT_DATE,S1213,RedundantThrowsDeclarationCheck,S00117,S00122,S1488,S1905,UselessImportCheck,DuplicatedBlocks,S1226,S00112,S1155,S00108,S1151,S1132,S1481
0,0,2005-11-28 03:15:35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,280,2005-11-29 01:34:22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,285,2005-11-29 02:25:03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,323,2005-11-29 03:11:23,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
4,347,2005-11-29 04:01:17,0,0,0,0,0,0,0,1,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625,121702,2012-02-17 22:50:39,17,142,8,0,34,12,68,155,18,110,10,3,2,52,15
1626,121706,2012-02-18 00:18:07,17,142,8,0,34,12,68,155,18,110,10,3,2,52,15
1627,121696,2012-02-18 00:18:30,17,142,8,0,34,12,68,155,18,110,10,3,2,52,15
1628,121712,2012-02-23 21:25:20,17,142,8,0,34,12,68,155,18,110,10,3,2,52,15


In [3]:
df = pd.read_csv("/Users/mrobredo23/OULU/PROMISE_TDpred2024/data/biweekly_data/archiva.csv")
df

,COMMIT_DATE,SQALE_INDEX,S1213,RedundantThrowsDeclarationCheck,S00117,S00122,S1488,S1905,UselessImportCheck,DuplicatedBlocks,S1226,S00112,S1155,S00108,S1151,S1132,S1481
0,2005-11-28 03:15:35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2005-12-12 08:09:34,4540.0,0.0,2.0,0.0,2.0,1.0,0.0,15.0,9.0,0.0,0.0,4.0,0.0,0.0,39.0,20.0
2,2005-12-27 02:34:36,6607.0,0.0,5.0,0.0,3.0,1.0,0.0,33.0,9.0,0.0,0.0,4.0,3.0,0.0,45.0,8.0
3,2006-01-15 00:30:24,5929.0,0.0,11.0,0.0,0.0,0.0,0.0,1.0,12.0,0.0,3.0,6.0,0.0,0.0,0.0,1.0
4,2006-01-30 02:51:46,7754.0,0.0,17.0,0.0,1.0,0.0,0.0,1.0,10.0,0.0,9.0,6.0,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2011-12-19 22:38:00,108851.0,17.0,142.0,7.0,0.0,24.0,12.0,68.0,136.0,18.0,108.0,10.0,3.0,2.0,41.0,15.0
132,2012-01-04 08:49:50,110243.0,17.0,142.0,8.0,0.0,34.0,12.0,68.0,145.0,18.0,112.0,10.0,3.0,2.0,52.0,15.0
133,2012-01-18 13:43:23,110861.0,17.0,142.0,8.0,0.0,34.0,12.0,68.0,146.0,18.0,112.0,10.0,3.0,2.0,52.0,15.0
134,2012-02-02 20:39:36,121632.0,17.0,142.0,8.0,0.0,34.0,12.0,70.0,152.0,18.0,110.0,10.0,3.0,2.0,52.0,15.0


In [14]:
robjects.r('/Users/mrobredo23/OULU/PROMISE_TDpred2024/codes/R_models.R')

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR